In [1]:
import numpy as np 
import pandas as pd 
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model



2024-05-31 22:02:03.631433: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-31 22:02:03.631504: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-31 22:02:03.632836: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-31 22:02:03.640717: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-31 22:02:04.496849: W tensorflow/compiler/tf2

In [2]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, MultiHeadAttention, LayerNormalization, Add, Dropout
from tensorflow.keras.models import Model
import tensorflow as tf


In [3]:
lines=pd.read_csv("../data/Hindi_English_Truncated_Corpus.csv",encoding='utf-8')

In [4]:
lines.head(20)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
5,tides,The then Governor of Kashmir resisted transfer...,कश्मीर के तत्कालीन गवर्नर ने इस हस्तांतरण का व...
6,indic2012,In this lies the circumstances of people befor...,इसमें तुमसे पूर्व गुज़रे हुए लोगों के हालात हैं।
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
8,indic2012,“”Global Warming“” refer to warming caused in ...,ग्लोबल वॉर्मिंग से आशय हाल ही के दशकों में हुई...
9,tides,You may want your child to go to a school that...,हो सकता है कि आप चाहते हों कि आप का नऋर्नमेनटे...


In [5]:
pd.isnull(lines).sum()

source              0
english_sentence    2
hindi_sentence      0
dtype: int64

In [6]:
lines=lines[~pd.isnull(lines['english_sentence'])]

In [7]:
lines.drop_duplicates(inplace=True)

In [8]:
lines=lines.sample(n=25000,random_state=42)
lines.shape

(25000, 3)

In [9]:
# Lowercase all characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())

In [10]:
# Remove quotes
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [11]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [12]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))


In [13]:
# Add start and end tokens to target sequences
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [14]:
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [15]:
len(all_eng_words)

30899

In [16]:
len(all_hindi_words)

36900

In [17]:
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [18]:
lines.head()

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
25520,indic2012,islam is word from arabic and it full word is ...,START_ इस्लाम शब्द अरबी भाषा का शब्द है जिसका ...,14,21
118633,ted,everything is reliant on these computers working,START_ इन कंप्यूटरों पर सब कुछ निर्भर है _END,7,9
113495,tides,parliament does not control the government,START_ संसद का सरकार पपर नियंत्रण नपहीं रहता _END,6,9
29783,tides,race equality new laws,START_ नये कानून नस्ली समानता _END,4,6
111804,tides,the provision would not affect the power of pa...,START_ व्यवसायों आदि से होने वाली आय के बारे म...,22,24


In [19]:
lines[lines['length_eng_sentence']>30].shape

(2463, 5)

In [20]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [21]:
lines.shape

(16512, 5)

In [22]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [23]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [24]:
print(max_length_src)

20


In [25]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(30899, 36900)

In [26]:
num_decoder_tokens += 1 #for zero padding


In [27]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [28]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [29]:
lines = shuffle(lines)
lines.head(10)

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
15414,indic2012,formation of the cabinet,START_ परिषद का गठन _END,4,5
58870,tides,checking keep the methods you use to check as ...,START_ जांच करना ः जांच के तरीकों को जितना सम्...,13,17
17652,indic2012,she was awarded for literature by padma bhusha...,START_ में भारत सरकार ने उनकी साहित्यिक सेवा क...,13,16
34538,indic2012,single the decision of which can be challenged...,START_ एकल जिसके निर्णय को उच्च न्यायालय की डि...,14,17
103268,indic2012,by the name of “”hindi tool“” installer of kus...,START_ कुशीनारा का हिन्दीटूल्स नामक इंस्टालर ज...,12,12
10032,ted,with the commandments held by the leader,START_ एक नेता के दिये गये निर्देशों के अनुसार...,7,11
70519,tides,according to the findings of the trial court t...,START_ अदालत के निष्कर्षों में भगत सिंह के विर...,20,15
28054,tides,these were the founders of the karnataka branc...,START_ ये ही कर्णाटक के कलचुरी कहलाए _END,12,8
49071,ted,you can walk across to another group,START_ अगर आपको अपना समूह पसंद न आये तो आप _END,7,11
65367,tides,but these hopes were not fulfilled,START_ किंतु ये आशायें पूर्ण नहीं हुई _END,6,8


### Split the data into train and test

In [30]:
X, y = lines['english_sentence'], lines['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((13209,), (3303,))

In [31]:
X_train

103865    maximum registered rainfall in a single day in...
46880     the second book named “”the odecity of hope“” ...
31524     controlled burn during spring cultivation in a...
30434       the prevailing colour of these animals is white
92018                       especially at “ zero interest ”
                                ...                        
30538          the white marble is used in its construction
23274                                        is he up to it
98138           akbars son jahangir built this mausoleum in
81127     like by a protein being destroyed by proteolys...
29978                        we can take away from cocacola
Name: english_sentence, Length: 13209, dtype: object

In [32]:
y_train

103865    START_ मुंबई में दर्ज एक दिन में सर्वोच्च वर्ष...
46880     START_ उनकी दूसरी पुस्तक द ओडेसिटी आफ होप अक्त...
31524     START_ दक्षिण जॉर्जिया के controlled burnएक खे...
30434     START_ इस नस्ल के ढोरों का सामान्य रंग सफेद हो...
92018                  START_ खासकर जब याज दर शून्य हो _END
                                ...                        
30538     START_ इसमें भी श्वेत संगमर्मर का प्रयोग हुआ ह...
23274                START_ क्या वे इसके लिए तैयार हैं _END
98138     START_ अकबर के पुत्र जहाँगीर ने इस मकबरे का नि...
81127     START_ जैसे प्रोटीन प्रोटेओलाइसिस से ध्वस्त हो...
29978            START_ जो हम कोकाकोला से सीख सकते हैं _END
Name: hindi_sentence, Length: 13209, dtype: object

In [33]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

In [34]:
def transformer_encoder(input_layer, head_size, num_heads, ff_dim, dropout=0):
    attn_output = MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(input_layer, input_layer)
    attn_output = Dropout(dropout)(attn_output)
    attn_output = Add()([input_layer, attn_output])
    attn_output = LayerNormalization(epsilon=1e-6)(attn_output)
    
    ff_output = Dense(ff_dim, activation='relu')(attn_output)
    ff_output = Dropout(dropout)(ff_output)
    ff_output = Dense(input_layer.shape[-1])(ff_output)
    ff_output = Add()([attn_output, ff_output])
    ff_output = LayerNormalization(epsilon=1e-6)(ff_output)
    
    return ff_output


In [35]:
def transformer_decoder(input_layer, encoder_output, head_size, num_heads, ff_dim, dropout=0):
    attn_output_1 = MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(input_layer, input_layer)
    attn_output_1 = Dropout(dropout)(attn_output_1)
    attn_output_1 = Add()([input_layer, attn_output_1])
    attn_output_1 = LayerNormalization(epsilon=1e-6)(attn_output_1)
    
    attn_output_2 = MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(attn_output_1, encoder_output)
    attn_output_2 = Dropout(dropout)(attn_output_2)
    attn_output_2 = Add()([attn_output_1, attn_output_2])
    attn_output_2 = LayerNormalization(epsilon=1e-6)(attn_output_2)
    
    ff_output = Dense(ff_dim, activation='relu')(attn_output_2)
    ff_output = Dropout(dropout)(ff_output)
    ff_output = Dense(input_layer.shape[-1])(ff_output)
    ff_output = Add()([attn_output_2, ff_output])
    ff_output = LayerNormalization(epsilon=1e-6)(ff_output)
    
    return ff_output


In [36]:
# Hyperparameters
head_size = 256
num_heads = 4
ff_dim = 512
num_encoder_tokens = len(all_eng_words)  # Replace with your actual vocabulary size
num_decoder_tokens = len(all_hindi_words)  # Replace with your actual vocabulary size
latent_dim = 256
dropout = 0.1

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(num_encoder_tokens, latent_dim, mask_zero=True)(encoder_inputs)
encoder_outputs = transformer_encoder(enc_emb, head_size, num_heads, ff_dim, dropout)

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero=True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_outputs = transformer_decoder(dec_emb, encoder_outputs, head_size, num_heads, ff_dim, dropout)

# Final Dense Layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the Model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')


2024-05-31 22:02:06.872568: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-31 22:02:06.909155: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-31 22:02:06.909405: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [37]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 256)            7910144   ['input_1[0][0]']             
                                                                                                  
 multi_head_attention (Mult  (None, None, 256)            1051904   ['embedding[0][0]',           
 iHeadAttention)                                                     'embedding[0][0]']           
                                                                                                  
 dropout (Dropout)           (None, None, 256)            0         ['multi_head_attention[0][

In [38]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 130
epochs = 60

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)



/tmp/ipykernel_15676/2713035456.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


Epoch 1/60


2024-05-31 22:02:10.161100: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-05-31 22:02:11.332403: I external/local_xla/xla/service/service.cc:168] XLA service 0x7ff72c07c750 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-31 22:02:11.332438: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-05-31 22:02:11.336217: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-31 22:02:11.346451: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1717173131.399731   15815 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


101/101 [==============================] - 57s 517ms/step - loss: 7.2937 - val_loss: 6.6359
Epoch 2/60
101/101 [==============================] - 51s 507ms/step - loss: 6.1363 - val_loss: 5.9681
Epoch 3/60
101/101 [==============================] - 51s 504ms/step - loss: 5.4545 - val_loss: 5.6511
Epoch 4/60
101/101 [==============================] - 51s 503ms/step - loss: 4.8581 - val_loss: 5.4022
Epoch 5/60
101/101 [==============================] - 51s 506ms/step - loss: 4.3049 - val_loss: 5.1101
Epoch 6/60
101/101 [==============================] - 51s 505ms/step - loss: 3.7906 - val_loss: 4.9630
Epoch 7/60
101/101 [==============================] - 51s 506ms/step - loss: 3.3142 - val_loss: 4.8323
Epoch 8/60
101/101 [==============================] - 51s 505ms/step - loss: 2.8652 - val_loss: 4.7249
Epoch 9/60
101/101 [==============================] - 51s 505ms/step - loss: 2.4560 - val_loss: 4.6224
Epoch 10/60
101/101 [==============================] - 51s 505ms/step - loss: 2.0774

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_outputs)
# Decoder setup for inference
decoder_inputs_inf = Input(shape=(None,))
dec_emb_inf = dec_emb_layer(decoder_inputs_inf)

# The decoder needs the encoder output for cross-attention
encoder_outputs_inf = Input(shape=(None, latent_dim))

# Transformer Decoder layers for inference
decoder_outputs_inf = transformer_decoder(dec_emb_inf, encoder_outputs_inf, head_size, num_heads, ff_dim, dropout)

# Final Dense layer for output generation
decoder_outputs_inf = decoder_dense(decoder_outputs_inf)

# Define the inference decoder model
decoder_model = Model(
    [decoder_inputs_inf, encoder_outputs_inf],
    decoder_outputs_inf
)



In [ ]:
def decode_sequence(input_seq):
    # Encode the input as context vectors (encoder output).
    encoder_output_value = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']
    
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        # Predict the next token
        output_tokens = decoder_model.predict([target_seq, encoder_output_value])
        
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' ' + sampled_char
        
        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
            len(decoded_sentence) > 50):
            stop_condition = True
        
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

    return decoded_sentence


In [ ]:

encoder_model.save('../models/english_to_hindi_translator/encoder_model_e2h_transformer.h5')
decoder_model.save('../models/english_to_hindi_translator/decoder_model_e2h_transformer.h5')


In [ ]:
import pickle

# Save the tokenizers
with open('../models/english_to_hindi_translator/english_tokenizer_e2h_transformer.pkl', 'wb') as f:
    pickle.dump(input_token_index, f)

with open('../models/english_to_hindi_translator/hindi_tokenizer_e2h_transformer.pkl', 'wb') as f:
    pickle.dump(target_token_index, f)

# Save the reverse tokenizers
with open('../models/english_to_hindi_translator/reverse_english_tokenizer_e2h.pkl', 'wb') as f:
    pickle.dump(reverse_input_char_index, f)

with open('../models/english_to_hindi_translator/reverse_hindi_tokenizer_e2h.pkl', 'wb') as f:
    pickle.dump(reverse_target_char_index, f)


In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import load_model
import pickle

# Load the models
encoder_model = load_model('../models/english_to_hindi_translator/encoder_model_e2h_transformer.h5')
decoder_model = load_model('../models/english_to_hindi_translator/decoder_model_e2h_transformer.h5')

# Load tokenizers and index mappings
with open('../models/english_to_hindi_translator/english_tokenizer_e2h.pkl', 'rb') as f:
    input_token_index = pickle.load(f)

with open('../models/english_to_hindi_translator/hindi_tokenizer_e2h.pkl', 'rb') as f:
    target_token_index = pickle.load(f)

with open('../models/english_to_hindi_translator/reverse_english_tokenizer_e2h.pkl', 'rb') as f:
    reverse_input_char_index = pickle.load(f)

with open('../models/english_to_hindi_translator/reverse_hindi_tokenizer_e2h.pkl', 'rb') as f:
    reverse_target_char_index = pickle.load(f)

max_length_src = 20  
latent_dim = 300
num_decoder_tokens = len(target_token_index) + 1

def translate_sentence(input_sentence):
    input_seq = [input_token_index.get(word, 0) for word in input_sentence.split()]
    input_seq = tf.keras.preprocessing.sequence.pad_sequences([input_seq], maxlen=max_length_src, padding='post')

    # Encode the input as context vectors (encoder output)
    encoder_output_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        # Predict the next token
        output_tokens = decoder_model.predict([target_seq, encoder_output_value])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index.get(sampled_token_index, '')

        # Exit condition: either hit max length or find stop character
        if sampled_char == '_END' or len(decoded_sentence) > max_length_src:
            stop_condition = True
        else:
            decoded_sentence.append(sampled_char)

        # Update the target sequence (of length 1)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

    return ' '.join(decoded_sentence)

# Example usage
print(translate_sentence("kya kar rha hai"))
